In [2]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import mat73
from collections import namedtuple
from scipy.signal import butter, sosfilt
from scipy.signal import stft

In [1]:
def get_eeg(filename):
    with h5py.File(filename) as h5f:
        print(h5f['eeg'].keys())
        return h5f['eeg/data'][:-1, :]

In [3]:
raw_data_folder = r'..\data\raw'
filename = 'mulines_OVERTexample.h5'
filename_full = os.path.join(raw_data_folder, filename)
eeg_1 = get_eeg(filename_full)
filename = 'mulines_NOMOVexample.h5'
filename_full = os.path.join(raw_data_folder, filename)
eeg_2 = get_eeg(filename_full)

<KeysViewHDF5 ['blocks', 'data', 'streamInfo']>
<KeysViewHDF5 ['blocks', 'data', 'streamInfo']>


In [14]:
MatParameters = namedtuple(
    "MatParameters",
    "eeg_chans_inds which_photo photo_chan_ind photo_analog_parameters spatialW fft_length fft_overlap fft_window features features_window features_average LDAW LDA_Const b a sos MdlA",
)

def load_parameters_from_mat(mat: str) -> MatParameters:
    mat = mat73.loadmat(mat)
    return MatParameters(**{f: mat.get(f, None) for f in MatParameters._fields})

In [21]:
mat = r'..\data\classifier_test.mat'
p = load_parameters_from_mat(mat)

In [80]:
features = np.array([[2.90911310e-04, 3.25784108e-04],
       [1.61114930e-04, 2.36121295e-04],
       [1.52268507e-04, 8.32761980e-05],
       [2.59683664e-04, 2.75827424e-04],])

In [81]:
# def apply_naiveAnatolyan(MdlA, feature):
MdlA = p.MdlA
posterior_prob = np.zeros(features.shape)
for ch in range(features.shape[1]):
    posterior_prob[:, ch] = np.interp(features[:, ch], MdlA['bin'][ch], MdlA['value'][ch])
posterior_prob = np.array(posterior_prob)
numerator = np.prod(posterior_prob, axis=1)
denominator = np.prod(posterior_prob, axis=1) + np.prod(1-posterior_prob, axis=1)
prob = numerator / denominator

array([[0.99645011, 1.        ],
       [0.99645626, 1.        ],
       [0.99645667, 1.        ],
       [0.99645159, 1.        ]])

In [65]:
feature.shape[1]

2

In [84]:
numerator = np.prod(posterior_prob, axis=1)
denominator = np.prod(posterior_prob, axis=1) + np.prod(1-posterior_prob, axis=1)
prob = numerator / denominator
prob

array([1., 1., 1., 1.])